In [ ]:
import pandas as pd
import json
import torch
import random
import torch
import string
from datasets import load_dataset, load_metric
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
from pathlib import Path
from pandas import json_normalize
from transformers import AutoTokenizer, AutoModel, AdamW
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
df = pd.read_csv("train_df.csv")

In [ ]:
df1 = df.astype(str).apply(lambda x: f"{x['answer']}, identifier: {x['identifier']}, sentence: {x['sentence']}", axis=1)

In [ ]:
df = pd.read_csv("train_df.csv")
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df1)

def retrieve_documents(query, tfidf_matrix, df):
    # Transform the query into the same vector space
    query_vector = vectorizer.transform([query])

    # Compute cosine similarities between the query and all documents
    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()

    # Get the top 5 most similar documents
    top_docs_indices = cosine_similarities.argsort()[:-10:-1]

    # Return the corresponding documents
    return df.iloc[top_docs_indices]

In [ ]:

model_name = "deepset/roberta-base-squad2"
nlp = pipeline("question-answering", model=model_name)

In [ ]:
question = "Which sentences treated immunity of a witnesses" # dataset["train"]["question"][6]

In [ ]:
result = nlp(question=question, context=retrieve_documents(question, tfidf_matrix, df1).str.cat(sep='. '))
print(result)

In [ ]:
import numpy as np
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric
import torch
nltk.download('punkt')

metric = load_metric("rouge")

tokenizer = AutoTokenizer.from_pretrained("t5-base")


def compute_metrics(eval_pred):
    decoded_preds, labels = eval_pred

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels[:len(decoded_preds)],
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
    #                   for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)

    metrics = {k: round(v, 4) for k, v in result.items()}

    return metrics

In [ ]:

# prepare dataloader
dataset = load_dataset("csv", data_files="test_ds.csv")
dataset["train"] = dataset["train"].shuffle().select(range(1000))
dataset.set_format(type='torch', columns=['question'])
dataloader = torch.utils.data.DataLoader(dataset["train"], batch_size=32)


# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
   print(batch)
   predictions = [nlp(question=question, context=retrieve_documents(question, tfidf_matrix, df1).str.cat(sep='. ')) for question in batch["question"]]
   print(predictions)
   all_predictions.append(predictions)
   print(i, end="\r")
   # progress_bar.add(1)



In [ ]:
# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(dataset["train"]["answer"], max_length=512,
                       truncation=True, padding="max_length")["input_ids"]
''
# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
predictions_labels[0] = [pred["answer"] for pred in predictions_labels[0]]
compute_metrics(predictions_labels)

In [ ]:
dataset["train"]["answer"]

In [ ]:
question = "What is the immunity of state and regional legislators?"
nlp(question=question, context=retrieve_documents(question, tfidf_matrix, df1).str.cat(sep='. '))

In [ ]:
predictions_labels